<a href="https://colab.research.google.com/github/Pandatoey/LAB229351/blob/main/%E0%B8%AA%E0%B8%B3%E0%B9%80%E0%B8%99%E0%B8%B2%E0%B8%82%E0%B8%AD%E0%B8%87_Lab02_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #2

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve

# For Fashion-MNIST
from tensorflow.keras.datasets import fashion_mnist

# For 20 Newsgroups
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## Part 1: Marketing Campaign Dataset - Manual Data Preprocessing & Logistic Regression

### Load the Marketing Campaign Dataset ([Data Information](https://archive.ics.uci.edu/dataset/222/bank+marketing))

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be (`'yes'`) or not (`'no'`) subscribed.

In [39]:
bank_url = 'https://raw.githubusercontent.com/donlap/ds352-labs/main/bank.csv'

df = pd.read_csv(bank_url, sep=';', na_values=['unknown'])
df = df.drop(["emp.var.rate", "cons.price.idx", "cons.conf.idx",	"euribor3m", "nr.employed"], axis=1)
print("Shape of the dataset:", df.shape)
df.head()

Shape of the dataset: (41188, 16)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,no
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,149,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,no


### Data Exploration

In [40]:
print("--- Missing Values Count ---")
print(df.isnull().sum())

--- Missing Values Count ---
age               0
job             330
marital          80
education      1731
default        8597
housing         990
loan            990
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
y                 0
dtype: int64


In [41]:
print("--- Unique Values for Categorical Columns ---")
for col in df.select_dtypes(include='object').columns:
    print(f"\n'{col}' unique values:")
    print(df[col].value_counts(dropna=False)) # Include NaN counts

--- Unique Values for Categorical Columns ---

'job' unique values:
job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
NaN                330
Name: count, dtype: int64

'marital' unique values:
marital
married     24928
single      11568
divorced     4612
NaN            80
Name: count, dtype: int64

'education' unique values:
education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
NaN                     1731
illiterate                18
Name: count, dtype: int64

'default' unique values:
default
no     32588
NaN     8597
yes        3
Name: count, dtype: int64

'housing' unique values:
housing
yes    21576
no     18622
NaN      990
Name: count, dtype: int64


### Data Preprocessing

In [42]:
# Map target variable 'y' to 0 (no) and 1 (yes)
df['y_new'] = df['y'].map({'yes':1,'no':0}) # Write your code here


# Drop 'duration' due to data leakage


# Define features (X) and target (y)
y = df['y_new']
x = df.drop(['y','y_new'], axis=1)

print(y)
print(x)
# Split the data BEFORE any transformations


# Print data shape



0        0
1        0
2        0
3        0
4        0
        ..
41183    1
41184    0
41185    0
41186    1
41187    0
Name: y_new, Length: 41188, dtype: int64
       age          job  marital            education default housing loan    contact month day_of_week  duration  campaign  pdays  previous     poutcome
0       56    housemaid  married             basic.4y      no      no   no  telephone   may         mon       261         1    999         0  nonexistent
1       57     services  married          high.school     NaN      no   no  telephone   may         mon       149         1    999         0  nonexistent
2       37     services  married          high.school      no     yes   no  telephone   may         mon       226         1    999         0  nonexistent
3       40       admin.  married             basic.6y      no      no   no  telephone   may         mon       151         1    999         0  nonexistent
4       56     services  married          high.school      no      n

In [43]:
x_train, x_test ,y_train , y_test = train_test_split(x,y,test_size = 0.3)
x_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
33298,51,blue-collar,married,basic.6y,NaN,yes,no,cellular,may,tue,211,1,999,1,failure
397,29,blue-collar,divorced,basic.4y,no,no,no,telephone,may,tue,231,1,999,0,nonexistent
20578,34,admin.,married,high.school,no,yes,no,cellular,aug,tue,228,5,999,0,nonexistent
18656,39,technician,married,professional.course,no,yes,no,cellular,jul,thu,23,5,999,0,nonexistent
35748,49,technician,married,high.school,NaN,no,no,cellular,may,mon,222,4,999,1,failure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18354,34,technician,married,university.degree,no,no,no,cellular,jul,thu,153,3,999,0,nonexistent
35254,26,services,married,high.school,no,yes,no,cellular,may,fri,140,9,999,1,failure
4869,35,technician,married,university.degree,NaN,no,no,telephone,may,wed,899,1,999,0,nonexistent
8394,26,blue-collar,single,high.school,no,yes,no,telephone,jun,tue,167,1,999,0,nonexistent


We will apply `StandardScaler()`, `OrdinalEncoder()`, and `OneHotEncoder()` on a few selected columns.

**1. Numerical Feature: `age` and `campaign` (Standard Scaling)**

In [ ]:
num_cols_demo = ['age', 'campaign']

scaler = StandardScaler()

# Fit the scaler ONLY on the training data


X_train_scaled_demo = # Write your code here
X_test_scaled_demo = # Write your code here

SyntaxError: invalid syntax (ipython-input-2180463677.py, line 8)

In [ ]:
num_cols_demo = ['age', 'campaign']

scaler = StandardScaler()

x_train[num_cols_demo] = scaler.fit_transform(x_train[num_cols_demo])
x_test[num_cols_demo] = scaler.fit_transform(x_test[num_cols_demo] )

In [ ]:
x_train.describe()

Let's take a look at the transformed `age` and `campaign` features and their statistics.

In [ ]:
print("\nOriginal X_train 'age' and 'campaign' head:")
print(x_train[num_cols_demo].head())
print("\nScaled X_train 'age' and 'campaign' head:")
print(pd.DataFrame(x_train[num_cols_demo], columns=num_cols_demo, index=x_train.index).head())

print("\nMean of scaled 'age' (train):", x_train[num_cols_demo][:, 0].mean())
print("Std Dev of scaled 'campaign' (train):", x_train[num_cols_demo][:, 1].std())

**2. Ordinal Feature: `education` (Ordinal Encoding with Imputation)**

- **Imputation**

In [ ]:
ord_col_demo = ['education']

imputer_ord = SimpleImputer(strategy='most_frequent')

## Write your code here

X_train_imputed_ord_demo = # Write your code here
X_test_imputed_ord_demo = # Write your code here

In [ ]:
ord_col_demo = ['education']

imputer_ord = SimpleImputer(strategy='most_frequent')

x_train[ord_col_demo] = imputer_ord.fit_transform(x_train[ord_col_demo])
x_test[ord_col_demo] = imputer_ord.transform(x_test[ord_col_demo])

x_train['education']

- **Ordinal Encoding**

In [ ]:
education_categories = [
    'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
    'professional.course', 'university.degree', 'masters', 'doctorate'
]

In [ ]:
ordinal_encoder = OrdinalEncoder(categories=[education_categories])

## Write your code here


X_train_ord_encoded_demo = # Write your code here
X_test_ord_encoded_demo = # Write your code here

In [ ]:
ordinal_encoder = OrdinalEncoder(categories=[education_categories])

x_train[ord_col_demo] = ordinal_encoder.fit_transform(x_train[ord_col_demo])
x_test[ord_col_demo] = ordinal_encoder.transform(x_test[ord_col_demo])

Let's take a look at the imputed and ordinal-encoded `education`.

In [ ]:
print("\nOriginal X_train 'education' head:")
print(X_train[ord_col_demo].iloc[20:25])
print("\nImputed X_train 'education' head (after imputer.transform):")
print(pd.DataFrame(X_train_imputed_ord_demo, columns=ord_col_demo, index=X_train.index).iloc[20:25])
print("\nOrdinal Encoded X_train 'education' head:")
print(pd.DataFrame(X_train_ord_encoded_demo, columns=ord_col_demo, index=X_train.index).iloc[20:25])

**3. Nominal Feature: `job` (One-Hot Encoding with Imputation)**

- **Imputation**

In [ ]:
nom_col_demo = ['job']

imputer_nom = SimpleImputer(strategy='most_frequent')
imputer_nom.fit(x_train[nom_col_demo])

x_train[nom_col_demo] = imputer_nom.transform(x_train[nom_col_demo])
x_test[nom_col_demo] = imputer_nom.transform(x_test[nom_col_demo])

- **Nominal Encoding**

In [ ]:
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

## Write your code here


X_train_onehot_encoded_demo = ## Write your code here
X_test_onehot_encoded_demo = ## Write your code here

In [ ]:
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

x_train_onehot = onehot_encoder.fit_transform(x_train[['job']])
x_test_onehot = onehot_encoder.fit_transform(x_test[['job']])

x_train_onehot.shape

In [ ]:
x_train_onehot

In [ ]:
x_train = pd.concat([x_train.reset_index(drop=True),
                     pd.DataFrame(x_train_onehot, columns = onehot_encoder.get_feature_names_out(['job']))] , axis = 1)

x_train

In [ ]:
x_test = pd.concat([x_test.reset_index(drop=True),
                     pd.DataFrame(x_test_onehot, columns = onehot_encoder.get_feature_names_out(['job']))] , axis = 1)

x_train = x_train.drop(['job'],axis=1)
x_test

In [ ]:
print("\nOriginal X_train 'job' head:")
print(X_train[nom_col_demo].iloc[40:45])
print("\nImputed X_train 'job' head (after imputer.transform):")
print(pd.DataFrame(X_train_imputed_nom_demo, columns=nom_col_demo, index=X_train.index).iloc[40:45])
print("\nOne-Hot Encoded X_train 'job' shape:", X_train_onehot_encoded_demo.shape)
print("First 5 rows of One-Hot Encoded X_train 'job':")
print(pd.DataFrame(X_train_onehot_encoded_demo, columns=onehot_encoder.get_feature_names_out(nom_col_demo), index=X_train.index).iloc[40:45])

### **Exercise 1: Apply All Preprocessing & Train Logistic Regression**

Now, it's your turn to apply these preprocessing steps to *all* relevant columns and then train a Logistic Regression model.

**Instructions:**

1.  Look at the Variable Table in [this link](https://archive.ics.uci.edu/dataset/222/bank+marketing).
2. Make lists for `numerical_features`, `ordinal_features`, and `nominal_features`.
3. Preprocess the features. It is safer to make a copy of `X_train` using:
   ```
   X_train_copy = X_train.copy()
   X_test_copy = X_test.copy()
   ```
   and preprocess `X_train_copy` instead.

   **For nominal features, concat the one-hot encoded features using [`pd.concat(..., axis=1)`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) and drop the old nominal features from the dataframe.**
4. Train Logistic Regression on the preprocessed `X_train_copy` and `y_train`.
5. Evaluate the Model:
    *   Make predictions on the preprocessed `X_test_copy`.
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?


In [44]:
# --- YOUR CODE FOR EXERCISE 1 STARTS HERE ---
x_train_copy = x_train.copy()
x_test_copy = x_test.copy()

In [45]:
x_train_copy

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
15650,26,admin.,married,high.school,no,yes,no,cellular,jul,mon,176,4,999,0,nonexistent
2868,36,technician,married,professional.course,no,yes,no,telephone,may,wed,160,2,999,0,nonexistent
16558,27,services,single,high.school,no,yes,yes,cellular,jul,wed,158,2,999,0,nonexistent
33302,28,blue-collar,married,basic.4y,no,no,no,cellular,may,tue,590,1,999,0,nonexistent
15129,41,entrepreneur,divorced,basic.9y,no,yes,yes,cellular,jul,thu,219,1,999,0,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10440,32,technician,married,professional.course,no,yes,no,telephone,jun,mon,226,2,999,0,nonexistent
9206,35,technician,single,university.degree,no,no,no,telephone,jun,fri,78,3,999,0,nonexistent
26544,39,technician,married,university.degree,no,yes,yes,cellular,nov,thu,42,1,999,0,nonexistent
18549,27,blue-collar,single,basic.4y,no,no,yes,cellular,jul,thu,83,1,999,0,nonexistent


In [46]:
x_test_copy

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
33298,51,blue-collar,married,basic.6y,NaN,yes,no,cellular,may,tue,211,1,999,1,failure
397,29,blue-collar,divorced,basic.4y,no,no,no,telephone,may,tue,231,1,999,0,nonexistent
20578,34,admin.,married,high.school,no,yes,no,cellular,aug,tue,228,5,999,0,nonexistent
18656,39,technician,married,professional.course,no,yes,no,cellular,jul,thu,23,5,999,0,nonexistent
35748,49,technician,married,high.school,NaN,no,no,cellular,may,mon,222,4,999,1,failure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18354,34,technician,married,university.degree,no,no,no,cellular,jul,thu,153,3,999,0,nonexistent
35254,26,services,married,high.school,no,yes,no,cellular,may,fri,140,9,999,1,failure
4869,35,technician,married,university.degree,NaN,no,no,telephone,may,wed,899,1,999,0,nonexistent
8394,26,blue-collar,single,high.school,no,yes,no,telephone,jun,tue,167,1,999,0,nonexistent


In [47]:
#numerical_features ตัวแปรเชิงปริมาณ
num_cols_demo = ['age','duration','campaign','pdays','previous']

scaler = StandardScaler()

x_train_copy[num_cols_demo] = scaler.fit_transform(x_train_copy[num_cols_demo])
x_test_copy[num_cols_demo] = scaler.fit_transform(x_test_copy[num_cols_demo] )

In [48]:
#ordinal_features ตัวแปรเรียงอันดับ
ord_col_demo = ['education']

imputer_ord = SimpleImputer(strategy='most_frequent')

x_train_copy[ord_col_demo] = imputer_ord.fit_transform(x_train_copy[ord_col_demo])
x_test_copy[ord_col_demo] = imputer_ord.transform(x_test_copy[ord_col_demo])

x_train_copy['education']

,education
15650,high.school
2868,professional.course
16558,high.school
33302,basic.4y
15129,basic.9y
...,...
10440,professional.course
9206,university.degree
26544,university.degree
18549,basic.4y


In [49]:
#ordinal encoding
education_categories = [
    'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown'
]

ordinal_encoder = OrdinalEncoder(categories=[education_categories])

x_train_copy[ord_col_demo] = ordinal_encoder.fit_transform(x_train_copy[ord_col_demo])
x_test_copy[ord_col_demo] = ordinal_encoder.transform(x_test_copy[ord_col_demo])

In [50]:
#nominal_features
nom_col_demo = ['job','marital','contact','poutcome']

imputer_nom = SimpleImputer(strategy='most_frequent')
imputer_nom.fit(x_train[nom_col_demo])

x_train_copy[nom_col_demo] = imputer_nom.transform(x_train_copy[nom_col_demo])
x_test_copy[nom_col_demo] = imputer_nom.transform(x_test_copy[nom_col_demo])

In [51]:
#nominal encoding
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

x_train_onehot = onehot_encoder.fit_transform(x_train_copy[['job','marital','contact','poutcome']])
x_test_onehot = onehot_encoder.transform(x_test_copy[['job','marital','contact','poutcome']])

x_train_onehot.shape

(28831, 19)

In [52]:
x_train_copy = x_train_copy.drop(columns=nom_col_demo)

x_test_copy = x_test_copy.drop(columns=nom_col_demo)

In [53]:
x_train_copy = x_train_copy.drop(['default','housing','loan','month','day_of_week'], axis=1)
x_test_copy = x_test_copy.drop(['default','housing','loan','month','day_of_week'], axis=1)

In [54]:
x_train_copy = pd.concat([x_train_copy.reset_index(drop=True),
                     pd.DataFrame(x_train_onehot, columns = onehot_encoder.get_feature_names_out(['job','marital','contact','poutcome']))] , axis = 1)
x_train_copy

,age,education,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_divorced,marital_married,marital_single,contact_cellular,contact_telephone,poutcome_failure,poutcome_nonexistent,poutcome_success
0,-1.342613,3.0,-0.317739,0.503404,0.195368,-0.34714,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,-0.383452,5.0,-0.379492,-0.209149,0.195368,-0.34714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2,-1.246697,3.0,-0.387211,-0.209149,0.195368,-0.34714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
3,-1.150781,0.0,1.280099,-0.565426,0.195368,-0.34714,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.096129,2.0,-0.151780,-0.565426,0.195368,-0.34714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28826,-0.767116,5.0,-0.124764,-0.209149,0.195368,-0.34714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
28827,-0.479368,6.0,-0.695972,0.147127,0.195368,-0.34714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
28828,-0.095703,6.0,-0.834914,-0.565426,0.195368,-0.34714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
28829,-1.246697,0.0,-0.676674,-0.565426,0.195368,-0.34714,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [55]:
x_test_copy = pd.concat([x_test_copy.reset_index(drop=True),
                     pd.DataFrame(x_test_onehot, columns = onehot_encoder.get_feature_names_out(['job','marital','contact','poutcome']))] , axis = 1)
x_test_copy

,age,education,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_divorced,marital_married,marital_single,contact_cellular,contact_telephone,poutcome_failure,poutcome_nonexistent,poutcome_success
0,1.048474,1.0,-0.181714,-0.567663,0.19552,1.689702,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,-1.064877,0.0,-0.104698,-0.567663,0.19552,-0.355117,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,-0.584570,3.0,-0.116251,0.924015,0.19552,-0.355117,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
3,-0.104263,5.0,-0.905664,0.924015,0.19552,-0.355117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.856351,3.0,-0.139355,0.551095,0.19552,1.689702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12352,-0.584570,6.0,-0.405060,0.178176,0.19552,-0.355117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
12353,-1.353061,3.0,-0.455121,2.415693,0.19552,1.689702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
12354,-0.488508,6.0,2.467634,-0.567663,0.19552,-0.355117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
12355,-1.353061,3.0,-0.351149,-0.567663,0.19552,-0.355117,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [56]:
y_train

,y_new
15650,0
2868,0
16558,0
33302,1
15129,0
...,...
10440,0
9206,0
26544,0
18549,0


In [57]:
x_train_copy

,age,education,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_divorced,marital_married,marital_single,contact_cellular,contact_telephone,poutcome_failure,poutcome_nonexistent,poutcome_success
0,-1.342613,3.0,-0.317739,0.503404,0.195368,-0.34714,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,-0.383452,5.0,-0.379492,-0.209149,0.195368,-0.34714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2,-1.246697,3.0,-0.387211,-0.209149,0.195368,-0.34714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
3,-1.150781,0.0,1.280099,-0.565426,0.195368,-0.34714,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.096129,2.0,-0.151780,-0.565426,0.195368,-0.34714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28826,-0.767116,5.0,-0.124764,-0.209149,0.195368,-0.34714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
28827,-0.479368,6.0,-0.695972,0.147127,0.195368,-0.34714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
28828,-0.095703,6.0,-0.834914,-0.565426,0.195368,-0.34714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
28829,-1.246697,0.0,-0.676674,-0.565426,0.195368,-0.34714,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [59]:
model = LogisticRegression()

model.fit(x_train_copy,y_train)

y_pred = model.predict(x_test_copy)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [60]:
target_names = ['without diabetes', 'with diabetes']
print(classification_report(y_test, y_pred, target_names=target_names))

                  precision    recall  f1-score   support

without diabetes       0.93      0.98      0.95     11011
   with diabetes       0.66      0.35      0.46      1346

        accuracy                           0.91     12357
       macro avg       0.79      0.67      0.71     12357
    weighted avg       0.90      0.91      0.90     12357



## Part 2: Fashion-MNIST Dataset - Image Classification

### Load Fashion-MNIST Dataset

The Fashion-MNIST dataset consists of 28x28 grayscale images of fashion items.

In [ ]:
(fm_X_train, fm_y_train), (fm_X_test, fm_y_test) = fashion_mnist.load_data()

print(f"Fashion-MNIST Train data shape: {fm_X_train.shape}")
print(f"Fashion-MNIST Train labels shape: {fm_y_train.shape}")
print(f"Fashion-MNIST Test data shape: {fm_X_test.shape}")
print(f"Fashion-MNIST Test labels shape: {fm_y_test.shape}")

In [ ]:
print(f"First image {fm_X_train[0]}")
print(f"First label {fm_y_train[0]}")

### Visualize Fashion-MNIST Images

Let's see what these images look like.

In [ ]:
fashion_mnist_class_names = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

# Visualize the images
## Write your code here



### **Exercise 2: Preprocessing Images (Flatten and Scale)**

Images are 2D arrays (matrices of pixels) and pixel values are integers from 0-255. For Logistic Regression, we need:
*  **Flattening:** Convert each 28x28 image into a 1D array of 784 features.
*  **Scaling:** Normalize pixel values from [0, 255] to [0, 1].

**Instructions:**

1.   **Flatten:** Use the `.reshape()` method (see [documentation](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.reshape.html)). For `fm_X_train_binary` (shape `(num_samples, 28, 28)`), you want to reshape it to `(num_samples, 28*28)`.
2.  **Scale:** Divide the flattened pixel values by 255.0 to get values between 0 and 1.
3.   **Train Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
    *   Fit the model on your *processed* `fm_X_train_scaled` and `fm_y_train`.
4.   **Make Predictions:** Use `predict()` to make predictions on the *processed* `fm_X_test_scaled`.
5.   **Print Classification Report:** Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?
6.   **Visualize Misclassifications:**
    *   Find the indices in `fm_X_test_binary` where your model made incorrect predictions (i.e., `fm_y_pred != fm_y_test`).
    *   Select 5 of these misclassified images.
    *   Plot these images (using `plt.imshow`). For each image, print its true label and its predicted label.

In [ ]:
# --- YOUR CODE FOR EXERCISE 2 STARTS HERE ---


## Part 3: 20 Newsgroups Dataset - Text Classification

### Load 20 Newsgroups Dataset

The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics.

In [ ]:
news_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
news_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)

X_train_news, y_train_news = news_train.data, news_train.target
X_test_news, y_test_news = news_test.data, news_test.target

print(f"Number of training documents: {len(X_train_news)}")
print(f"Number of test documents: {len(X_test_news)}")
print(f"Categories: {news_train.target_names}")

### Explore Sample Document

In [ ]:
# Print the first document and its class
## Write your code here



### Preprocessing: Text Vectorization Demonstration with `TfidfVectorizer`

$$
\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)
$$

Where:

$$
\text{TF}(t, d) = \frac{\text{number of word }t\text{ in } d}{\text{number of words in } d} \quad \text{ and } \quad
\text{IDF}(t, D) = \log\left(\frac{\text{total number of documents}}{\text{number of documents that contain word }t}\right).
$$

In [ ]:
sample_sentences = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?"
]

vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the sample sentences
sample_vec_output_sparse = # Write your code here

sample_vec_output_dense = sample_vec_output_sparse.toarray()

print(vectorizer.vocabulary_)
print(vectorizer.get_feature_names_out())
print(sample_vec_output_dense)

### **Exercise 3: Apply TF-IDF Vectorization to Full Dataset**

Now, apply `TfidfVectorizer` to the actual training and testing datasets for the 20 Newsgroups classification task.

**Instructions:**

1.  **Initialize `TfidfVectorizer`:**
    *   Initialize `TfidfVectorizer`. Use `stop_words='english'` to remove common words.
2.  **Fit and Transform Training Data:**
    *   Call `fit_transform()` on `X_train_news` to learn the vocabulary and transform the training text into TF-IDF features. Store the result in `X_train_vec`.
3.  **Transform Test Data:**
    *   Call `transform()` on `X_test_news` using the *already fitted* vectorizer. Store the result in `X_test_vec`. **Crucially, do not call `fit_transform()` on the test data!** This would cause data leakage.
4.  **Initialize Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
5.  **Train the Model:**
    *   Fit the model on your `X_train_vec` and `y_train_news`.
6.  **Make Predictions:**
    *   Make predictions using `predict()` on the `X_test_vec`.
7.  **Evaluate the Model:**
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?

In [ ]:
# --- YOUR CODE FOR EXERCISE 3 STARTS HERE ---


